<a href="https://colab.research.google.com/github/thunguyen177/lda_pca/blob/master/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let $A$ be the matrix that contains the eigenvectors of $W^{-1}B$ in the descending order of the corresponding eigenvalues. Let

$Y=XA$ 

then the $(i,j)$-position of $Y$ is equal to $X_{i.}A_{j.}$, which is the $j^{th}$ discriminant for observation $X_{i.}$

$Y=\begin{pmatrix}Y_1\\Y_2\\\vdots\\Y_s\end{pmatrix}$ has  mean $\mu_{iY}$ under population $\pi_k$

We allocate observation $x$ to $\pi_k$ if 

$(y-\mu_{kY})'(y-\mu_{kY})\le (y-\mu_{iY})'(y-\mu_{iY}) \forall i\neq k$

In [0]:
import numpy as np

def find_mean_vectors(X, y):
    labels = np.unique(y)
    num_classes = labels.shape[0]
    mean_vectors = []
    for l in labels:
        mean_vectors.append(np.mean(X[y==l], axis=0))
    return mean_vectors

def within(X, y):
    labels = np.unique(y)
    num_classes = labels.shape[0]
    num_features = X.shape[1]
    mean_vectors = find_mean_vectors(X, y)
    W = np.zeros((num_features, num_features))
    for label, mv in zip(labels, mean_vectors):
        class_sc_mat = np.zeros((num_features, num_features))                 
        for row in X[y == label]:
            row, mv = row.reshape(num_features, 1), mv.reshape(num_features, 1)
            class_sc_mat += (row-mv).dot((row-mv).T)
        W += class_sc_mat                           
    return W

def between(X, y):
    overall_mean = np.mean(X, axis=0)
    num_features = X.shape[1]
    mean_vectors = find_mean_vectors(X, y)    
    B = np.zeros((num_features, num_features))
    for i, mean_vec in enumerate(mean_vectors):  
        n = X[y==i+1,:].shape[0]
        mean_vec = mean_vec.reshape(num_features, 1)
        overall_mean = overall_mean.reshape(num_features, 1)
        B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)
    return B


In [0]:
def predict(X,y,Xtest,ytest):
    W, B = within(X, y), between(X, y)
    eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(W).dot(B))
    ids = np.argsort(eig_vals)[::-1]
    A = eig_vecs[ids,] 
    r = (eig_vals != 0).sum()
    labels = np.unique(y)
    mean_vectors = find_mean_vectors(X, y)
    g =labels.size
    A = eig_vecs[ids[range(r)]]  
    predicted_labels = []
    for idx in range(Xtest.shape[0]):
      x = Xtest[idx,:]
      p = []
      s = []
      for i in range(g):
        pr = []
        for j in range(r):
          pr = np.append([pr],[(A[j].dot(x-mean_vectors[i]))**2])
        s = np.append([s],[pr.sum()])
      predicted_labels = np.append([predicted_labels],[labels[np.argmin(s)]])  
    num_correct = (predicted_labels == ytest).sum()
    correct_rate = num_correct/float(ytest.size)
    
    return predicted_labels, correct_rate

In [111]:
mean1 = [0, 0]
mean2 = [3,4]
mean3 = [5,5]
cov = [[1, 0], [0, 2]]  # diagonal covariance
def generate_data(n):
  x1 = np.random.multivariate_normal(mean1, cov, n)
  x2 = np.random.multivariate_normal(mean2, cov, n)
  x3 = np.random.multivariate_normal(mean3, cov, n)
  X = np.concatenate((x1, x2,x3), axis=0)
  y = np.concatenate([np.repeat(1,n),np.repeat(2,n),np.repeat(3,n)])
  return X,y

X,y = generate_data(30)
Xtest,ytest = generate_data(20)

predicted_labels, correct_rate = predict(X,y,Xtest,ytest)
predicted_labels

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1.,
       1., 2., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2., 2., 2.,
       3., 2., 1., 3., 2., 2., 3., 3., 2., 3., 3., 2., 3., 3., 2., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 2., 3.])

In [112]:
correct_rate

0.8333333333333334

# Fisher discriminant analysis:
\begin{align*}
B &= \sum_{i=1}^gn_i(\bar{x}_i-\bar{x})(\bar{x}_i-\bar{x})'\\
W &=  \sum_{i=1}^g(n_i-1)S_i
\end{align*}

Let $\hat{\lambda}_1,...,\hat{\lambda}_s$ be the increasing nonzero eigenvalues of $W^{-1}B$ and $\hat{a}_1,...,\hat{a}_s$ be their eigenvectors, respectively.

Let $r$ be the number of nonzero eigenvalues of $W^{-1}B$ .
Then, $x\rightarrow \pi_k$ if $s_k=\sum_{j=1}^r[a'_j(x-\bar{x}_k)]^2\le (s_i=)\sum_{j=1}^r[a'_j(x-\bar{x}_i)]^2\forall i\neq k$ 
